In [1]:
import random

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, utils, models

from hex import board, player, game
import util
import move_evaluator
import move_generator

In [2]:
games = pd.read_csv("finaldataframe.csv")

In [4]:
games_over_2 = games[games["move_list"].str.len() > 4]

boards, winners = util.makeInputOutputTotal(games_over_2["move_list"].values[:10000],
                                            games_over_2["winner"].values[:10000])

In [ ]:
len(boards)

In [ ]:
p1 = player.ModelPlayer(move_generator.SamplingProbabilisticGenerator(10), move_evaluator.TestMoveEvaluator())
p2 = player.ModelPlayer(move_generator.ExhaustiveProbabilisticGenerator(), move_evaluator.TestMoveEvaluator())
new_moves, new_winners =  util.generate_games(1000, p1, p2)

In [5]:
model = tf.keras.Sequential([
    layers.Conv2D(256, kernel_size=(3, 3),
                  activation='relu',
                  input_shape=(13, 13, 1)),
    layers.Conv2D(256, (3, 3), activation ="relu"),
    layers.MaxPool2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPool2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(2, activation='softmax')])

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
for i in range(5):
    batch_boards =  boards[i * 10000 : i * 10000 + 10000]
    batch_winners = winners[i * 10000 : i * 10000 + 10000]
    model.fit(batch_boards.reshape(list(batch_boards.shape) + [1]), batch_winners, epochs=1)


Epoch 1/1
10000/10000 [==============================] - 76s 8ms/step - loss: 0.4267 - acc: 0.7794
Epoch 1/1
10000/10000 [==============================] - 96s 10ms/step - loss: 0.3198 - acc: 0.8499
Epoch 1/1
10000/10000 [==============================] - 77s 8ms/step - loss: 0.3046 - acc: 0.8562
Epoch 1/1
10000/10000 [==============================] - 70s 7ms/step - loss: 0.3033 - acc: 0.8704
Epoch 1/1
10000/10000 [==============================] - 77s 8ms/step - loss: 0.2716 - acc: 0.8753


In [ ]:
test_boards, test_winners = util.makeInputOutputTotal(games_over_2["move_list"].values[10000:10500],
                                            games_over_2["winner"].values[10000:10500])

In [ ]:
model.evaluate(test_boards.reshape(list(test_boards.shape) + [1]), test_winners)

In [ ]:
model.save("blackwins_model.h5")

In [ ]:
model = models.load_model("blackwins_model.h5")

In [ ]:
p1 = player.ModelPlayer(move_generator.SamplingGenerator(10), move_evaluator.BlackWinMoveEvaluator(model))
p2 = player.ModelPlayer(move_generator.SamplingGenerator(10), move_evaluator.BlackWinMoveEvaluator(model))
new_moves, new_winners = util.generate_games(20, p1, p2)

In [ ]:
new_winners

In [ ]:
wins = pd.Series(new_winners)
len(wins[wins == "black"])

In [ ]:
new_boards, new_winners = util.makeInputOutputTotal(new_moves, new_winners, scraped=False)